In [ ]:
import pickle
import torch
import numpy as np
from codes.query_solver import GeometricSolver

EMBEDDING_DIM = 512
MODEL_PATH = "/home/cc/phd/KGEmbeddings/models/TransE_FB15k_0/"

with open('queries-set2.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

queries = loaded_dict['queries']
results = loaded_dict['results']

qs = GeometricSolver(MODEL_PATH, EMBEDDING_DIM, k_neighbors=50, k_results=25, device='cuda')

In [ ]:
idx = np.random.randint(len(queries))

qs.set_k(k_neighbors=100, k_results=15)
res = qs.execute_query(queries[idx], proj_mode="inter", agg_mode="union")

founds = [ f for f in res if f in results[idx][1] ]
print(f"Query ({idx}): {queries[idx]}")
print(f"Found: {len(founds)} on {len(results[idx][1])} targets from {len(res)} results")